In [0]:
%sql
-- 1. Krijimi i Database-it
CREATE DATABASE IF NOT EXISTS movies_db;
USE movies_db;

In [0]:
%python
storage_key = dbutils.secrets.get(scope="movie-scope", key="storage-access-key")

spark.conf.set(
    "fs.azure.account.key.errisbinxhija01.dfs.core.windows.net",
    storage_key
)

In [0]:
%sql
DROP TABLE IF EXISTS recommendations_bronze;
DROP TABLE IF EXISTS ratings_history_bronze;
DROP TABLE IF EXISTS movies_bronze;
DROP TABLE IF EXISTS belief_data_bronze;


In [0]:
%sql
DROP TABLE IF EXISTS recommendations_silver;
DROP TABLE IF EXISTS belief_silver;
DROP TABLE IF EXISTS movies_silver;
DROP TABLE IF EXISTS ratings_silver;

In [0]:
%python

base_path = "abfss://movies-data@errisbinxhija01.dfs.core.windows.net/bronze/"

tables_info = {
    "movies_bronze": "movies.csv",
    "recommendations_bronze": "user_recommendation_history.csv",
    "belief_data_bronze": "belief_data.csv",
    "ratings_history_bronze": "user_rating_history.csv"
}

for table_name, file_name in tables_info.items():
    # 1. Leximi i skedarit CSV
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(f"{base_path}{file_name}")
    
    # 2. Ruajtja si tabelë Delta në shtresën Bronze
    # Kjo krijon tabelën nëse nuk ekziston ose e mbishkruan nëse po
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"movies_db.{table_name}")
    
    print(f"✅ Tabela {table_name} u krijua me {df.count()} rreshta.")

In [0]:
%sql
SHOW TABLES;

In [0]:

%sql
SELECT 
    COUNT(*) as total_rows
FROM recommendations_bronze;

In [0]:
%sql
SELECT 
    m.title, 
    r.rating, 
    r.tstamp
FROM ratings_history_bronze r
JOIN movies_bronze m ON r.movie_id = m.movieId
LIMIT 10;

In [0]:
%sql
DESCRIBE movies_db.movies_bronze

In [0]:
%sql
DESCRIBE recommendations_bronze

In [0]:
%sql
DESCRIBE ratings_history_bronze

In [0]:
%sql
DESCRIBE belief_data_bronze